# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-10724000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-6131000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = '5.AK93.AKJT72.T4'

# the auction goes:
auction = ["PAD_START","PASS", "PASS", "2D"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, False)

bid = bot_bid.bid(auction)
print(bid.to_dict()['candidates'])

# what's your bid?

[{'call': '3D', 'insta_score': 0.905, 'expected_score': -38, 'adjustment': 45}, {'call': 'X', 'insta_score': 0.053, 'expected_score': -24, 'adjustment': 3}]


In [3]:
bid.samples

['x.AK9x.AKJTxx.Tx AQTx.Jxxx.x.98xx J9xxx.QT8x..KQxx K8x.x.Q98xxx.AJx 0.95925 \n P-P-2D-3D-P-4S-P-P-P (7.1)  \n P-P-2D-X-P-2S-P-3D-P-3H-P-P-P (9.8) ',
 'x.AK9x.AKJTxx.Tx QJxxx.JT.9.Q8xxx AT8xx.Q8xxx..Kxx K9.xx.Q8xxxx.AJ9 0.95912 \n P-P-2D-3D-P-4S-P-P-P (7.9)  \n P-P-2D-X-P-4S-P-P-P (7.9) ',
 'x.AK9x.AKJTxx.Tx Qxxx.Qxxx..KJ8xx AJT8x.8xx.x.Qxxx K9x.JT.Q98xxx.A9 0.95401 \n P-P-2D-3D-P-3S-P-3N-P-P-P (7.6)  \n P-P-2D-X-P-2S-P-3D-P-4S-P-P-P (6.5) ',
 'x.AK9x.AKJTxx.Tx AQxx.QTxxx.x.9xx JT98x.Jxx..KQJ8x Kxx.8.Q98xxx.Axx 0.94990 \n P-P-2D-3D-P-4S-P-P-P (7.5)  \n P-P-2D-X-P-2S-P-3D-P-4S-P-P-P (7.5) ',
 'x.AK9x.AKJTxx.Tx AQJx.JT8x.x.98xx K8xxx.Qxxx..Qxxx T9x.x.Q98xxx.AKJ 0.94889 \n P-P-2D-3D-P-3S-P-3N-P-P-P (7.2)  \n P-P-2D-X-P-2S-P-3D-P-3H-P-P-P (8.7) ',
 'x.AK9x.AKJTxx.Tx J98xxx.QTxx..Axx QTxx.J8x.9.KQ8xx AK.xx.Q8xxxx.J9x 0.94756 \n P-P-2D-3D-P-3S-P-3N-P-P-P (7.1)  \n P-P-2D-X-P-2S-P-3D-P-4S-P-P-P (5.8) ',
 'x.AK9x.AKJTxx.Tx QT9x.J8xx.x.K9xx KJ8xxx.Qxxx..QJ8 Ax.T.Q98xxx.Axxx 0.94267 \n P-P-2D-3

each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [4]:
# we got some samples above. if we want more, we can always get more

sample_hands, sorted_score, p_hcp, p_shp = bot_bid.sample_hands_auction(["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"])

for i in range(sample_hands.shape[0]):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])}')

x.AK9x.AKJTxx.Tx 98xxx.Q8xx.8xx.8 AKx.x.xx.AQJxxxx QJTx.JTxx.Q9.K9x
x.AK9x.AKJTxx.Tx Q9xxx.QTxx.8x.9x AJx..9xxx.AKQ8xx KT8x.J8xxx.Q.Jxx
x.AK9x.AKJTxx.Tx 98xxx.8xx.xx.8xx KTx.T.Q9xx.AKQ9x AQJx.QJxxx.8.Jxx
x.AK9x.AKJTxx.Tx AT9xx.QT.xx.Q98x KQ8.x.Q8xx.AKxxx Jxxx.J8xxxx.9.Jx
x.AK9x.AKJTxx.Tx KQ8xx.xxx.8x.Qxx AJT.x.Q9x.AK8xxx 9xxx.QJT8x.xx.J9
x.AK9x.AKJTxx.Tx KJ9x.Jxxxx.Qx.xx A8x.Q.9x.AKJ8xxx QTxxx.T8x.8xx.Q9
x.AK9x.AKJTxx.Tx Qxx.Qxx.Q8xxx.Jx AKx.JTxx.x.AK9xx JT98xx.8x.9.Q8xx
x.AK9x.AKJTxx.Tx KQ8xx.xx.9xxx.8x AJx.JT8x.Q.AQJxx T9xx.Qxx.8x.K9xx
x.AK9x.AKJTxx.Tx 98xxx.Qxxx.9.Kxx AKJ.x.xxxx.AQ9xx QTxx.JT8x.Q8.J8x
x.AK9x.AKJTxx.Tx KQTxx.Txx.Qxx.xx AJ9.QJxx.x.AKJxx 8xxx.8x.98x.Q98x
